In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [3]:
import pickle
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch

from src.problem import get_model, get_soc, load_instance

In [5]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))
instances

[PosixPath('../data/raw/97_9.jl'),
 PosixPath('../data/raw/97_9_1.jl'),
 PosixPath('../data/raw/97_9_10.jl'),
 PosixPath('../data/raw/97_9_2.jl'),
 PosixPath('../data/raw/97_9_3.jl'),
 PosixPath('../data/raw/97_9_4.jl'),
 PosixPath('../data/raw/97_9_5.jl'),
 PosixPath('../data/raw/97_9_6.jl'),
 PosixPath('../data/raw/97_9_7.jl'),
 PosixPath('../data/raw/97_9_8.jl'),
 PosixPath('../data/raw/97_9_9.jl')]

In [14]:
instance = load_instance(instances[1])

model = get_model(list(range(9)), instance, coupling=True)

model.setParam('TimeLimit', 60)
model.update()

model.optimize()

model.ObjVal, model.MIPGap, model.Runtime

(2493.0, 0.036903329322101885, 60.02215886116028)

In [20]:
%%timeit

instance = load_instance(instances[1])

model = get_model(list(range(9)), instance, coupling=True, new_ineq=True)

model.setParam('TimeLimit', 60)
model.update()

model.optimize()

model.ObjVal, model.MIPGap, model.Runtime

2.85 s ± 44.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
X = np.array([v.X for v in model.getVars()])
soc_vars_mask = np.array(['soc' in v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
soc = X[soc_vars_mask]
X = X[~soc_vars_mask]

# check that get_soc is working
soc_hat = get_soc(torch.from_numpy(X[None,:]), instance)
torch.isclose(soc_hat[0][1:], torch.from_numpy(soc)).all()

(1746,)

In [6]:
instance = load_instance(instances[3])

model = get_model(list(range(9)), instance, coupling=True)

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

opts[instances[3].name]['sol'].shape

(1746,)

In [7]:
from src.dataset import InstanceEarlyFixingDataset
from src.net import InstanceGCN

net = InstanceGCN(2, readout_op=None)

ds = InstanceEarlyFixingDataset(
    [load_instance(i) for i in instances],
    [opts[i.name]['sol'] for i in instances],
    samples_per_problem=10,
)

g1, y1 = ds[0]
g2, y2 = ds[13]

g = dgl.batch((g1, g2))

net(g).shape

torch.Size([2, 1746])